# Building OSM Full History objects from History Files

> Test case: Boulder, CO

In [31]:
import osmium as o
import numpy as np
import sys, os, json, pprint
from collections import Counter

main runtime, unfortuantely it's single threaded.

In [32]:
class FileHandler(o.SimpleHandler):                                                                                           
    def __init__(self):
        o.SimpleHandler.__init__(self)                                                                                              
        self.w_cnt = 0
        self.n_cnt = 0

    def node(self, n):
        '''
            Only save nodes which have extra attributes...
            
            TODO: If the first version of a node has 0 tags, but version 2 has tags, then version 1 may not show up in the history... maybe?
        '''
        self.n_cnt +=1
        if n.id in nodes:
            nodes[n.id].add(n)
        elif len(n.tags)>0:
            nodes[n.id] = Node(n)
            
        #Cache these locations:
        if n.id not in node_locations:
            node_locations[n.id] = []

        node_locations[n.id].append({
                'v':n.version,
                'c':n.changeset,
                'g':[n.location.lon, n.location.lat]
            })
        
        if self.n_cnt%10000==0:
            sys.stderr.write("\r{0} nodes processed".format(self.n_cnt))

    def way(self, w):
        self.w_cnt += 1
        if w.id in ways: 
            ways[w.id].add(w)
        else:
            ways[w.id] = Way(w)
        if self.w_cnt%1000==0:
            sys.stderr.write("\r{0} ways processed".format(self.w_cnt))

In [33]:
class OSMObject:
    def __init__(self, w):
        self.id = w.id
        self.history = []
        self.add(w)
        
    def add(self,w):
        self.history.append(
            {
                'version':w.version,
#                 'deleted':w.deleted, # These are not doing their part
#                 'visible':w.visible, #TODO: Why don't these work properly?
                
                'uid' : w.uid,
                'user': w.user,
                'created_at': w.timestamp.isoformat(),
                'timestamp' : w.timestamp,
                
                'tags': dict( (t.k, t.v) for t in list(w.tags) ),
                'geometry': self.get_geometry(w)
            })
        
    def process_history(self):
        if len(self.history[0]['tags']):
            self.history[0]['new_tags'] = self.history[0]['tags']
            
        if len(self.history)>1:

            #Ensure it's in order
            self.history.sort(key=lambda x: x['version'])
            
            for prev_idx, w in enumerate(self.history[1:]):

                prev_keys  = set(self.history[prev_idx]['tags'])
                these_keys = set(w['tags'])

                new_keys = these_keys - prev_keys
                if len(new_keys) > 0:
                    w['new_tags'] = dict( (k, w['tags'][k]) for k in new_keys)

                del_keys = prev_keys - new_keys - these_keys
                if len(del_keys) > 0:
                    w['deleted_tags'] = dict( (k, self.history[prev_idx]['tags'][k]) for k in del_keys)
                
                changed_tags = {}
                for k in these_keys:
                    if k in prev_keys:
                        if w['tags'][k] != self.history[prev_idx]['tags'][k]:
                            changed_tags[k] = (self.history[prev_idx]['tags'][k], w['tags'][k])
                if len(changed_tags):
                    w['changed_tags'] = changed_tags
                    
                w['seconds_since_last_edit'] = int((w['timestamp'] - self.history[prev_idx]['timestamp']).total_seconds())
            
    def as_geojson(self):
        geojson = {'type':'Feature'}
        geojson['properties']   = self.history[-1]['tags'].copy()
        geojson['properties']['@id']          = self.id
        geojson['properties']['@created_at']  = self.history[-1]['created_at']
        geojson['properties']['@uid']         = self.history[-1]['uid']
        geojson['properties']['@user']        = self.history[-1]['user']
        geojson['properties']['@version']     = self.history[-1]['version']

        for hist_obj in self.history:
            if 'timestamp' in hist_obj:
                del hist_obj['timestamp']
        
        if len(self.history)>1:    
            geojson['properties']['@object_history'] = self.history
        
        geojson['geometry'] = self.history[-1]['geometry']
        
        return geojson
    
    def __str__(self):
        """
            Override str() function so when calling print(), we get back the full information
        """
        string = "ID: {0}; revisions: {1}".format(self.id, len(self.history)-1)
        if len(self.history)>1:
            string += "\n-------------------------------------------------------------------------------"
            for o in self.history:
                string += "\n({0}) - {1} - {2}, Nodes: {3}".format(o['version'], o['user'], o['created_at'], len(o['geometry']['coordinates']))
                if 'new_tags' in o:
                    string += "\n\tNew Tags: {0}".format(o['new_tags'])
                if 'deleted_tags' in o:
                    string += "\n\tDeleted Tags: {0}".format(o['deleted_tags'])
                if 'changed_tags' in o:
                    string += "\n\tChanged Tags: {0}".format(o['changed_tags'])
            string += "\n==============================================================================="
        return string
            
class Way(OSMObject):
    def __init__(self, w):
        self.errors = 0
        OSMObject.__init__(self, w)
    
    def get_geometry(self,w):
        coords = []
        for n in w.nodes:
            #If there are multiple versions, then we sort and take the latest
            if len(node_locations[n.ref]) > 1:
                #Sort by changeset id
                node_locations[n.ref].sort(key=lambda x: x['c'])
                
                # Due to silly error from JOSM or Potlatch, we have to get hacky with this...
                try:
                    c = [x for x in node_locations[n.ref] if not x['c'] > w.changeset][-1]
                except:
                    #just take the first one
                    c = node_locations[n.ref][0]
                coords.append(c['g'])
            else:
                coords.append(node_locations[n.ref][0]['g'])
            
        return {"type":"LineString", "coordinates": coords}
#         try:
#             geom = o.geom.WKBFactory.create_linestring(w.nodes)
#         except Exception as e:
#             self.errors += 1
#             print(context(e))
#             sys.exit(1)
        
class Node(OSMObject):
    def __init__(self, n):
        self.errors = 0
        OSMObject.__init__(self, n)
    
    def get_geometry(self, n):
        return {"type": "Point", "coordinates":[ n.location.lon, n.location.lat]}

Run the handler to creat the objects

## Now process the object

In [34]:
node_locations = {}
nodes = dict({})
ways  = dict({})
h = FileHandler()
h.apply_file('/data/osm/boulder.osh.pbf', locations=True)

32000 ways processed

In [35]:
np.random.choice(list(node_locations.values()),10)

array([ [{'v': 1, 'g': [-105.2697989, 40.0016279], 'c': 18394600}, {'v': 2, 'g': [-105.2697976, 40.0016288], 'c': 18488376}],
       [{'v': 1, 'g': [-105.2731312, 40.0127697], 'c': 9367533}],
       [{'v': 1, 'g': [-105.2575574, 39.9985619], 'c': 14636030}, {'v': 2, 'g': [-105.2575692, 39.998568], 'c': 22880465}],
       [{'v': 1, 'g': [-105.2806574, 40.0126512], 'c': 18386564}, {'v': 2, 'g': [-105.2806596, 40.0126532], 'c': 18487988}],
       [{'v': 1, 'g': [-105.2394603, 39.9977948], 'c': 14577446}],
       [{'v': 1, 'g': [-105.2487372, 40.0046433], 'c': 22649897}],
       [{'v': 1, 'g': [-105.2493102, 40.0095958], 'c': 15213723}],
       [{'v': 1, 'g': [-105.2436587, 40.0130771], 'c': 15530040}],
       [{'v': 1, 'g': [-105.2773009, 40.009189], 'c': 18389204}, {'v': 2, 'g': [-105.2772982, 40.0091898], 'c': 18488099}],
       [{'v': 1, 'g': [-105.231968, 40.013479], 'c': 345684}, {'v': 2, 'g': [-105.231968, 40.013479], 'c': 3391217}]], dtype=object)

In [36]:
# Choose sample of data and process it!
sample_ways  = np.random.choice(list(ways.keys()),10000)
sample_nodes = np.random.choice(list(nodes.keys()),10000)

for way in [ways[w_id] for w_id in sample_ways]:
    way.process_history()
    
for node in [nodes[n_id] for n_id in sample_nodes]:
    node.process_history() 

In [37]:
with open('sample_boulder_history.geojsonl','w') as outFile:
    for way in [ways[w_id] for w_id in sample_ways]:
        outFile.write(json.dumps(way.as_geojson())+"\n")
    
    for node in [nodes[n_id] for n_id in sample_nodes]:
        outFile.write(json.dumps(node.as_geojson())+"\n")


In [146]:
# for w_id, way in ways.items():
for way in [ways[w_id] for w_id in first_ten_ways]:
    way.process_history()
#     way.inspect_changes()
    print(way.as_geojson())
    
# # for w_id, way in ways.items():
for node in [nodes[n_id] for n_id in first_ten_nodes]:
    node.process_history()
#     node.inspect_changes()

{'properties': {'highway': 'residential', 'created_by': 'YahooApplet 1.0', '@user': 'erinn', 'name': 'Colorado Avenue', '@created_at': datetime.datetime(2007, 3, 3, 1, 44, 57, tzinfo=datetime.timezone.utc), '@id': 4325621, '@version': 1, '@uid': 5879}, 'geometry': {'coordinates': [[-105.2664213, 40.0082102], [-105.2693825, 40.0081773]], 'type': 'LineString'}, 'type': 'Feature'}
{'properties': {'highway': 'residential', 'created_by': 'YahooApplet 1.0', '@user': 'erinn', 'name': '18th Street', '@created_at': datetime.datetime(2007, 3, 3, 1, 45, 15, tzinfo=datetime.timezone.utc), '@id': 4325622, '@version': 1, '@uid': 5879}, 'geometry': {'coordinates': [[-105.2693825, 40.0081773], [-105.2695327, 40.0080623]], 'type': 'LineString'}, 'type': 'Feature'}
{'properties': {'highway': 'service', '@uid': 1778953, '@created_at': datetime.datetime(2016, 3, 1, 1, 58, 30, tzinfo=datetime.timezone.utc), 'maxspeed': '10 mph', '@id': 4325623, '@version': 8, 'object_history': [{'new_tags': {'highway': 're

In [174]:
def date_handler(obj):
    if hasattr(obj, 'isoformat'):
        return obj.isoformat()
    else:
        return str(obj)

In [8]:
import numpy as np

In [9]:
len(np.unique(h.ids))

1433017

In [10]:
len(h.ids)

2764902

In [11]:
h.cnt

2764902

# Can we do stuff with the geometries?

In [241]:
from shapely.geometry import Point, LineString

In [242]:
from shapely.geometry import mapping
import json

In [243]:
points = [Point(p.lon, p.lat) for p in h.geometries]

In [244]:
json.dumps({"type":"GeometryCollection", "geometries" : [mapping(p) for p in points[:1000]]})

'{"geometries": [{"type": "Point", "coordinates": [33.4100813, 45.2611148]}, {"type": "Point", "coordinates": [33.4127985, 45.2647191]}, {"type": "Point", "coordinates": [33.4152544, 45.2676245]}, {"type": "Point", "coordinates": [33.4167175, 45.2692058]}, {"type": "Point", "coordinates": [33.4187554, 45.2708974]}, {"type": "Point", "coordinates": [33.4226744, 45.2743541]}, {"type": "Point", "coordinates": [33.4256529, 45.2770751]}, {"type": "Point", "coordinates": [33.4294674, 45.2806417]}, {"type": "Point", "coordinates": [33.4326026, 45.2833624]}, {"type": "Point", "coordinates": [33.4336477, 45.2846492]}, {"type": "Point", "coordinates": [33.4345883, 45.2861565]}, {"type": "Point", "coordinates": [33.4372009, 45.2902372]}, {"type": "Point", "coordinates": [33.4384028, 45.2920752]}, {"type": "Point", "coordinates": [33.4397091, 45.2936558]}, {"type": "Point", "coordinates": [33.4409109, 45.2949424]}, {"type": "Point", "coordinates": [33.4424263, 45.2962289]}, {"type": "Point", "coor